In [1]:
import random
import numpy as np
import json
from tqdm import tqdm
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import linear_sum_assignment
from collections import Counter
import time
import os
import pandas as pd
from scipy.signal import correlate
np. set_printoptions(threshold=np. inf)

In [2]:
date = 'Sept26th-'
filename = ''.join((date,'scan_stats.json'))
filename

'Sept26th-scan_stats.json'

In [3]:
import scan
import simulate_data

In [4]:
default_params = {
    'M':8,
    'N':40,
    'D':51,
    'T':1000,
    'seed':0,
    'num_SM_events':16,
    'SM_total_spikes':10,
    'noise':100
}
scan_dict = {
    'M':[1,4,8,16,32],
    'N':[10,20,40,80,120],
    'D':[11,31,51,71,101],
    'T':[1000],
    'seed':[0],
    'num_SM_events':[2,4,8,16,32],
    'SM_total_spikes':[3,5,10,20,50],
    'noise':[0,50,100,500,1000]
}

In [5]:
from scipy.signal import correlate
def get_acc(ground_truths,detected_patterns):
    # Calculate cross-correlation matrix
    cross_corr_matrix = np.zeros((ground_truths.shape[2], detected_patterns.shape[2]))
    SM_acc = np.zeros((ground_truths.shape[2]))
    
    if len(detected_patterns) == 0:
        return SM_acc, cross_corr_matrix
    
    for ground_truths_idx in range(ground_truths.shape[2]):
        for detected_patterns_idx in range(detected_patterns.shape[2]):
            cross_corr = np.zeros((ground_truths.shape[1]+detected_patterns.shape[1]-1))
            for n in range(ground_truths.shape[0]):
                cross_corr += correlate(ground_truths[n, :, ground_truths_idx], detected_patterns[n, :, detected_patterns_idx], mode='full')
            max_corr = np.max(cross_corr) / max(np.sum(ground_truths[...,ground_truths_idx]),np.sum(detected_patterns[...,detected_patterns_idx]))
            cross_corr_matrix[ground_truths_idx, detected_patterns_idx] = max_corr
#     print(cross_corr_matrix)
#     print( np.sum(ground_truths[...,ground_truths_idx]))
    SM_acc = np.max(cross_corr_matrix,axis=1)
    return SM_acc, cross_corr_matrix

In [6]:
param_combinations = []
for param_name, param_values in scan_dict.items():
    for param_value in param_values:
        params = default_params.copy()
        params[param_name] = param_value
        param_combinations.append(params)

In [8]:
len(param_combinations)

32

In [ ]:
import numpy as np
import random
import os
import pandas as pd
import json
from tqdm import tqdm

# Define default parameters and scan values
default_params = {
    'M': 8,
    'N': 40,
    'D': 51,
    'T': 1000,
    'seed': 0,
    'num_SM_events': 16,
    'SM_total_spikes': 10,
    'noise': 100
}

scan_dict = {
    'M': [1, 4, 8, 16, 32],
    'N': [10, 20, 40, 80, 120],
    'D': [11, 31, 51, 71, 101], 
    'num_SM_events': [2, 4, 8, 16, 32],
    'SM_total_spikes': [3, 5, 10, 20, 50],
    'noise': [0, 50, 100, 500, 1000]
}

# Generate all parameter combinations
param_combinations = []
for param_name, param_values in scan_dict.items():
    for param_value in param_values:
        params = default_params.copy()
        params[param_name] = param_value
        param_combinations.append(params)

num_samples = len(param_combinations)
results = []

# Iterate through parameter combinations
for idx, params in tqdm(enumerate(param_combinations)):

    if os.path.isfile(filename):
        df = pd.read_json(filename)
        with open(filename, 'r') as results_file:
            results = json.load(results_file)
    start = time.time()
    A_dense, A_sparse, B_dense, B_sparse, K_dense, K_sparse = simulate_data.generate_synthetic_data(params)
    pattern_template, sublist_keys_filt, window_time, cluster_time, sequence_time = scan.scan_raster(A_sparse[1], A_sparse[0], window_dim=params['D'])
    end = time.time()
    result = {
        'M':params['M'],
        'N':params['N'],
        'D':params['D'],
        'T':params['T'],
        'num_SM_events':params['num_SM_events'],
        'SM_total_spikes':params['SM_total_spikes'],
        'noise':params['noise'],
        'window_time': window_time,
        'cluster_time': cluster_time,
        'sequence_time': sequence_time,
        'total_time': end-start
    }

    results.append(result)

    with open(filename, 'w') as results_file:
        json.dump(results, results_file, indent=4)


0it [00:00, ?it/s]

260 Windows


1it [00:00,  1.33it/s]

2 patterns found...s... 50% 10.01 | opt_cutoff - 1.37 | most_detections - 15
734 Windows
6 patterns found...s... 83% 10.01 | opt_cutoff - 0.47000000000000003 | most_detections - 15tections - 155


2it [00:02,  1.48s/it]

...

2567 Windows
18 patterns found...... 94% 10.01 | opt_cutoff - 0.56 | most_detections - 15tections - 153detections - 13


26it [06:08, 15.92s/it]

6008 Windows
142 patterns found..... 99% 10.01 | opt_cutoff - 0.67 | most_detections - 6etections - 6t_detections - 1


27it [10:24, 88.03s/it]

40000 Windows


In [9]:
param_combinations

[{'M': 1,
  'N': 40,
  'D': 51,
  'T': 1000,
  'seed': 0,
  'num_SM_events': 16,
  'SM_total_spikes': 10,
  'noise': 100},
 {'M': 4,
  'N': 40,
  'D': 51,
  'T': 1000,
  'seed': 0,
  'num_SM_events': 16,
  'SM_total_spikes': 10,
  'noise': 100},
 {'M': 8,
  'N': 40,
  'D': 51,
  'T': 1000,
  'seed': 0,
  'num_SM_events': 16,
  'SM_total_spikes': 10,
  'noise': 100},
 {'M': 16,
  'N': 40,
  'D': 51,
  'T': 1000,
  'seed': 0,
  'num_SM_events': 16,
  'SM_total_spikes': 10,
  'noise': 100},
 {'M': 32,
  'N': 40,
  'D': 51,
  'T': 1000,
  'seed': 0,
  'num_SM_events': 16,
  'SM_total_spikes': 10,
  'noise': 100},
 {'M': 8,
  'N': 10,
  'D': 51,
  'T': 1000,
  'seed': 0,
  'num_SM_events': 16,
  'SM_total_spikes': 10,
  'noise': 100},
 {'M': 8,
  'N': 20,
  'D': 51,
  'T': 1000,
  'seed': 0,
  'num_SM_events': 16,
  'SM_total_spikes': 10,
  'noise': 100},
 {'M': 8,
  'N': 40,
  'D': 51,
  'T': 1000,
  'seed': 0,
  'num_SM_events': 16,
  'SM_total_spikes': 10,
  'noise': 100},
 {'M': 8,
  'N

In [47]:
num_samples = 3
param_combinations = np.array(np.meshgrid(*scan_dict.values())).T.reshape(-1, len(scan_dict))
num_iterations = len(param_combinations)

# Generate random indices for sampling
random_indices = random.sample(range(num_iterations), num_samples)
results = []


# Iterate through parameter combinations
for idx in tqdm(random_indices):
    
    params = {key: int(val) for key, val in zip(scan_dict.keys(), param_combinations[idx])}
    if os.path.isfile(filename):
        df = pd.read_json(filename)
        with open(filename, 'r') as results_file:
            results = json.load(results_file)
            
            
    A_dense, A_sparse, B_dense, B_sparse, K_dense, K_sparse = simulate_data.generate_synthetic_data(params)
    pattern_template, sublist_keys_filt, window_time, cluster_time, sequence_time = scan.scan_raster(A_sparse[1], A_sparse[0], window_dim=params['D'])

    print(params)


    result = {
        'idx':idx,
        'window_time':window_time,   
        'cluster_time':cluster_time,
        'sequence_time':sequence_time
    }

    results.append(result)

    with open(filename, 'w') as results_file:
        json.dump(results, results_file, indent=4)
    
    
    

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

1186 Windows
4 patterns found...s... 75% 10.01 | opt_cutoff - 0.67 | most_detections - 3etections - 3


 33%|████████████████████████████                                                        | 1/3 [00:02<00:04,  2.15s/it]

364 Windows


 67%|████████████████████████████████████████████████████████                            | 2/3 [00:02<00:01,  1.28s/it]

1 patterns found...s... 0%
120000 Windows


 67%|████████████████████████████████████████████████████████                            | 2/3 [00:11<00:05,  5.97s/it]

KeyboardInterrupt: 

In [46]:
params['D']

11

In [33]:
params

{'M': 16,
 'N': 10,
 'D': 11,
 'T': 1000,
 'seed': 0,
 'num_SM_events': 8,
 'SM_total_spikes': 5,
 'noise': 1000}

In [48]:
df = pd.read_json(filename)

In [49]:
df.head()

,idx,test,window_time,cluster_time,sequence_time
0,6090,test,NaN,NaN,NaN
1,12262,test,NaN,NaN,NaN
2,6635,test,NaN,NaN,NaN
3,15176,NaN,0.860845,1.173667,0.001995
4,4466,NaN,0.059833,0.575803,0.000000


In [51]:
N = 1000
M = 10
D = 71
T = 1000
seed=0

num_SM_events = 10
SM_total_spikes = 150
noise = 1000

params = {
    'N':N,
    'M':M,
    'D':D,
    'T':T,
    'seed':seed,
    'num_SM_events':num_SM_events,
    'SM_total_spikes':SM_total_spikes,
    'noise':noise
}

start = time.time()
A_dense, A_sparse, B_dense, B_sparse, K_dense, K_sparse = simulate_data.generate_synthetic_data(params)
pattern_template, sublist_keys_filt, window_time, cluster_time, sequence_time = scan.scan_raster(A_sparse[1], A_sparse[0], window_dim=params['D'])
end = time.time()
print(end-start)

15867 Windows
38 patterns found...... 97% 10.01 | opt_cutoff - 0.38 | most_detections - 19tections - 199detections - 7


KeyboardInterrupt: 